In [1]:
import os
import sys
sys.path.append('../input/pytorch-images-seresnet')

import os
import math
import time
import random
import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter

import scipy as sp
import numpy as np
import pandas as pd

from tqdm.auto import tqdm
from functools import partial

import cv2
from PIL import Image

from matplotlib import pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
import albumentations
from albumentations import *
from albumentations.pytorch import ToTensorV2


import timm

from torch.cuda.amp import autocast, GradScaler

import warnings 
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
BATCH_SIZE = 64
# BATCH_SIZE_b5 = 32
TEST_PATH = '../input/ranzcr-clip-catheter-line-classification/test'

In [3]:
test = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/sample_submission.csv')

In [4]:
class TestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['StudyInstanceUID'].values
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        file_path = f'{TEST_PATH}/{file_name}.jpg'
        image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
        mask = image > 0
        image = image[np.ix_(mask.any(1), mask.any(0))]

        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
#         image = cv2.imread(file_path)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        return image

In [5]:
def get_transforms(image_size=684):
        return Compose([
            Resize(image_size, image_size),
#             RandomResizedCrop(image_size, image_size, scale=(0.85, 1.0)),
#             HorizontalFlip(p=0.5),
            Normalize(),
            ToTensorV2(),
        ])

In [6]:
class ResNet200D(nn.Module):
    def __init__(self, model_name='resnet200d_320'):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=False)
        n_features = self.model.fc.in_features
        self.model.global_pool = nn.Identity()
        self.model.fc = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(n_features, 11)

    def forward(self, x):
        bs = x.size(0)
        features = self.model(x)
        pooled_features = self.pooling(features).view(bs, -1)
        output = self.fc(pooled_features)
        return output

class SeResNet152D(nn.Module):
    def __init__(self, model_name='seresnet152d'):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=False)
        n_features = self.model.fc.in_features
        self.model.global_pool = nn.Identity()
        self.model.fc = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(n_features, 11)

    def forward(self, x):
        bs = x.size(0)
        features = self.model(x)
        pooled_features = self.pooling(features).view(bs, -1)
        output = self.fc(pooled_features)
        return output
    
class RANZCRResNet200D(nn.Module):
    def __init__(self, model_name='resnet200d', out_dim=11):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=False)
        n_features = self.model.fc.in_features
        self.model.global_pool = nn.Identity()
        self.model.fc = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(n_features, out_dim)

    def forward(self, x):
        bs = x.size(0)
        features = self.model(x)
        pooled_features = self.pooling(features).view(bs, -1)
        output = self.fc(pooled_features)
        return output
    
class CustomEffNet(nn.Module):
    def __init__(self, model_name='tf_efficientnet_b5_ns', out_dim=11):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=False)
        n_features = self.model.classifier.in_features
        self.model.global_pool = nn.Identity()
        self.model.classifier = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Linear(n_features, 11)

    def forward(self, x):
        bs = x.size(0)
        features = self.model(x)
        pooled_features = self.pooling(features).view(bs, -1)
        output = self.classifier(pooled_features)
        return output

In [7]:
def inference(models, test_loader, device):
    tk0 = tqdm(enumerate(test_loader), total=len(test_loader))
    probs = []
    for i, (images) in tk0:
        images = images.to(device)
        avg_preds = []
        for model in models:
            with torch.no_grad():
                y_preds1 = model(images)
                y_preds2 = model(images.flip(-1))
            y_preds = (y_preds1.sigmoid().to('cpu').numpy() + y_preds2.sigmoid().to('cpu').numpy()) / 2
            avg_preds.append(y_preds)
        avg_preds = np.mean(avg_preds, axis=0)
        probs.append(avg_preds)
    probs = np.concatenate(probs)
    return probs

def tta_inference(model, loader, tta_steps=5):
    all_probs = []
    for i, step in enumerate(range(tta_steps)):
        probs = []
        for step, (images) in tqdm(enumerate(loader), total=len(loader)):
            images = images.to(device)
            with torch.no_grad():
                y_preds = model(images)
            y_preds = y_preds.sigmoid().to('cpu').numpy()
            probs.append(y_preds)
        all_probs.append(np.concatenate(probs))
    avg_probs = np.mean(all_probs, axis=0)

    return avg_probs

In [8]:
models200d_320_tuned_LowestLoss = []

model = ResNet200D()
model.load_state_dict(torch.load('../input/stage3-res320-f2e1-9734/Stage3_resnet200d_320_fold_2_epoch_1_97.34.pth'))
model.eval()                                                                             # loss 0.0936
model.to(device)
models200d_320_tuned_LowestLoss.append(model)

model = ResNet200D()
model.load_state_dict(torch.load('../input/stage3-resnet320-f3-96969738/Stage3_resnet200d_320_fold_3_epoch_2_97.38.pth'))
model.eval()                                                                            # loss  0.0984
model.to(device)
models200d_320_tuned_LowestLoss.append(model)


models200d_320_tuned = []

model = ResNet200D()
model.load_state_dict(torch.load("../input/stage3-resnet200d-320-f0e2-9718/Stage3_resnet200d_320_fold_3_epoch_2_97.18.pth"))
model.eval()                                                                                # loss 0.1034
model.to(device)
models200d_320_tuned.append(model)

# model = ResNet200D()
# model.load_state_dict(torch.load('../input/stage3-res320-f1e3-9726/Stage3_resnet200d_320_fold_1_epoch_3_97.26.pth'))
# model.eval()                                                                              # loss 0.1092
# model.to(device)
# models200d_320_tuned.append(model)

model = ResNet200D()
model.load_state_dict(torch.load('../input/stage3-resnet320-f3-96969738/Stage3_resnet200d_320_fold_3_epoch_1_96.96000000000001.pth'))
model.eval()                                                                                # loss 0.1031
model.to(device)
models200d_320_tuned.append(model)


models200d_320_general = []

model = ResNet200D()
model.load_state_dict(torch.load('../input/stage3-general-res320-f3-96719702/Stage3_resnet200d_320_fold_3_epoch_11_97.02.pth'))
model.eval()                                                                                # loss  0.1087
model.to(device)
models200d_320_general.append(model)

In [9]:
models200D_2_tuned = []
model = RANZCRResNet200D()
model.load_state_dict(torch.load('../input/resent200d-all-folds-fine-tuned/Stage3_resnet200d_fold_0_epoch_3_96.47.pth'))
model.eval()
model.to(device)
models200D_2_tuned.append(model)

model = RANZCRResNet200D()
model.load_state_dict(torch.load('../input/resent200d-all-folds-fine-tuned/Stage3_resnet200d_fold_1_epoch_2_95.88.pth'))
model.eval()
model.to(device)
models200D_2_tuned.append(model)

model = RANZCRResNet200D()
model.load_state_dict(torch.load('../input/resent200d-all-folds-fine-tuned/Stage3_resnet200d_fold_2_epoch_2_96.16.pth'))
model.eval()
model.to(device)
models200D_2_tuned.append(model)

model = RANZCRResNet200D()
model.load_state_dict(torch.load('../input/resent200d-all-folds-fine-tuned/Stage3_resnet200d_fold_3_epoch_4_96.26.pth'))
model.eval()
model.to(device)
models200D_2_tuned.append(model)

model = RANZCRResNet200D()
model.load_state_dict(torch.load('../input/resent200d-all-folds-fine-tuned/Stage3_resnet200d_fold_4_epoch_4_96.61999999999999.pth'))
model.eval()
model.to(device)
models200D_2_tuned.append(model)

SeResNet152D

In [10]:
models152D = []
# model = SeResNet152D()
# model.load_state_dict(torch.load('../input/seresnet152d-cv9615/seresnet152d_320_CV96.15.pth')['model'])
# model.eval()
# model.to(device)
# models152D.append(model)

# # model152d_96_16 = []
# model = SeResNet152D()
# model.load_state_dict(torch.load('../input/stage3-seresnet-f0e2-9616/Stage3_seresnet152d_fold_0_epoch_2_96.16.pth'))
# model.eval()
# model.to(device)
# models152D.append(model)

model = SeResNet152D()
model.load_state_dict(torch.load('../input/stage3-seresnet-f4e2-9669/Stage3_seresnet152d_fold_4_epoch_2_96.69.pth'))
model.eval()
model.to(device)
models152D.append(model)

## MAIN

In [11]:
test_dataset_864 = TestDataset(test, transform=get_transforms(image_size=864))
test_loader_864 = DataLoader(test_dataset_864, batch_size=BATCH_SIZE, shuffle=False, num_workers=4 , pin_memory=True)


predictions200d_320_tuned_LowestLoss = inference(models200d_320_tuned_LowestLoss, test_loader_864, device)
predictions200d_320_tuned = inference(models200d_320_tuned, test_loader_864, device)
predictions200d_320_general = inference(models200d_320_general, test_loader_864, device)
predictions200d_2_tuned = inference(models200D_2_tuned, test_loader_864, device)
predictions152d = inference(models152D, test_loader_864, device)

predictions = (1.20 * predictions200d_320_tuned_LowestLoss + predictions200d_320_tuned +
               0.70 * predictions200d_320_general + 0.75 * predictions200d_2_tuned + 0.75 * predictions152d) / 4.40

  0%|          | 0/56 [00:00<?, ?it/s]

  0%|          | 0/56 [00:00<?, ?it/s]

  0%|          | 0/56 [00:00<?, ?it/s]

  0%|          | 0/56 [00:00<?, ?it/s]

  0%|          | 0/56 [00:00<?, ?it/s]

In [12]:
target_cols = test.iloc[:, 1:12].columns.tolist()
test[target_cols] = predictions
test[['StudyInstanceUID'] + target_cols].to_csv('submission.csv', index=False)
test.head()

,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present
0,1.2.826.0.1.3680043.8.498.46923145579096002617...,0.041852,0.432320,0.594534,0.004390,0.010207,0.041273,0.950605,0.101350,0.154116,0.805482,0.975796
1,1.2.826.0.1.3680043.8.498.84006870182611080091...,0.000104,0.000349,0.000686,0.000397,0.000503,0.000326,0.000443,0.020665,0.027712,0.942575,0.000088
2,1.2.826.0.1.3680043.8.498.12219033294413119947...,0.000148,0.000470,0.000989,0.000823,0.000594,0.000390,0.000465,0.031381,0.392529,0.444609,0.000304
3,1.2.826.0.1.3680043.8.498.84994474380235968109...,0.008024,0.049029,0.124498,0.046826,0.034339,0.840853,0.071085,0.031741,0.159976,0.839155,0.014702
4,1.2.826.0.1.3680043.8.498.35798987793805669662...,0.000406,0.002102,0.003588,0.002307,0.001825,0.000745,0.003652,0.006509,0.041200,0.950859,0.000118
